In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Reference: lesson 5.9 and Microsoft website at:
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling( 
    {
    '--C': choice(0.0001, 0.001, 0.01, 0.1, 1,10,100,1000),
    '--max_iter': choice(100, 200, 300, 400, 500)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

script_folder = "./training"    
    
# Reference: lesson 6.3: copying the training file into the script folder
shutil.copy('./train.py', script_folder)
    

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = script_folder, 
              compute_target=cpu_cluster, 
              entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est, 
                          hyperparameter_sampling=ps,
                          policy=policy, 
                          primary_metric_name="Accuracy", 
                          primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                          max_total_runs = 30)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config = hyperdrive_config)

#reference: lesson 6.3: running a history widget to show the progress
runDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output = True)
assert(hyperdrive_run.get_status() == "Completed")


In [ ]:
import joblib 

# Get your best run and save the model from that run.
#reference lesson 6.3: finding and registering best model

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print(parameter_values)
print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('C:',parameter_values[3])
print('Max_Iter:',parameter_values[5])



In [ ]:
# Save the model
best_run.download_file("/outputs/model.joblib", "Logistic_reg_hd_bankmarketing.joblib")
#model = best_run.register_model(model_name = 'Logistic_reg_hd_bankmarketing', model_path = 'outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
import pandas as pd

# Because we are doing k-fold cross validation, there is no need to split the data 
# into train and test sets.

# The training_data parameter should contain both training features and a label column 
training_data = pd.concat([x, y], axis = 1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="Accuracy",
    training_data=training_data,
    label_column_name = y,
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [2]:
# Submit your automl run

exp = Experiment(workspace=ws, name="AutoML-project")
automl_run = exp.submit(config = automl_config)

# Reference: lesson 6.3: running a history widget to show the progress
runDetails(automl_run).show()
automl_run.wait_for_completion(show_output = True)
assert(automl_run.get_status() == "Completed")


In [ ]:
# Retrieve and save your best automl model.
# Reference: https://knowledge.udacity.com/questions/357775
automl_best_run, fitted_automl_model = automl_run.get_output() 
print(fitted_automl_model) 

In [ ]:
# Reference: https://knowledge.udacity.com/questions/357775
#Save the model
joblib.dump(fitted_automl_model, "/outputs/fitted_automl_model.joblib")
automl_best_run.download_file("/outputs/fitted_automl_model.joblib", "fitted_automl_model.joblib")
#model = automl_best_run.register_model(model_name = 'Logistic_reg_hd_bankmarketing', model_path = 'outputs/model')

In [ ]:
cpu_cluster.delete()
